In [1]:
path='F:/'
import numpy as np
file1=open(path+"jointsFile","rb")
file2 = open(path+"levelFile", "rb")
joints_array= np.load(file1)
labels= np.load(file2)
print(joints_array.shape)
print(labels.shape)


(11082, 72, 25, 3)
(11082, 100)


In [2]:
#Angel between Right shoulder(2) , Right Elbow(3) , Right Wrist(4)
#Angel between Left shoulder(5) , Left Elbow(6) , Left Wrist(7)

#Angel between Right Hip(9) , Right knee(10) , Right ankle(11)
#Angel between Left Hip(12) , Left knee(13) , Left ankle(14)
#Angel between Mid Hip(8) , Right knee10 , Right ankle(11)

#Distance between Left Ankle(14) And Right Ankle(11)
#Distance between Left Wrist(7) And Right Wrist(4)
import numpy as np
import math


def cal_angel(joints_1,joints_2,joints_3):
          #print(joints_1,joints_2)

          a = joints_1 - joints_2
          b = joints_2 - joints_3

          cosine_angle = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
          angle = np.arccos(cosine_angle)
          np.degrees(angle)
          #print(angle)
          return angle

def cal_dis(joints_1,joints_2):

          distance=math.sqrt( ((joints_2[0]-joints_1[0])**2)+((joints_2[1]-joints_1[1])**2)+((joints_2[2]-joints_1[2])**2) )
          #print(distance)
          return distance



length=joints_array.shape[0]

cols=72
row=7 
new_fe = [[[0] * cols] * row]*length
for j in range(length):
 
    #print(arr[0][2])
    for i in range(cols): 
        
        new_fe[j][0][i]= cal_angel(joints_array[j][i][2],joints_array[j][i][3],joints_array[j][i][4])
    for i in range(cols): 
        new_fe[j][1][i]= cal_angel(joints_array[j][i][5],joints_array[j][i][6],joints_array[j][i][7])
    for i in range(cols): 
        new_fe[j][2][i]= cal_angel(joints_array[j][i][9],joints_array[j][i][10],joints_array[j][i][11])
    for i in range(cols): 
        new_fe[j][3][i]= cal_angel(joints_array[j][i][12],joints_array[j][i][13],joints_array[j][i][14])
    for i in range(cols): 
        new_fe[j][4][i]= cal_angel(joints_array[j][i][8],joints_array[j][i][10],joints_array[j][i][11])
    for i in range(cols): 
        new_fe[j][5][i]= cal_dis(joints_array[j][i][14],joints_array[j][i][11])
    for i in range(cols): 
        new_fe[j][6][i]= cal_dis(joints_array[j][i][7],joints_array[j][i][4])
new_fe = np.asarray(new_fe)
print(new_fe.shape) 



<ipython-input-2-194ee4de89f6>:20: RuntimeWarning: invalid value encountered in float_scalars
  cosine_angle = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


(11082, 7, 72)


In [16]:
  with open('F:/'+ 'featcherFile', 'wb') as f:
        pickle.dump(new_fe, f)

In [3]:
#Import the libraries
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
(trainX, testX, trainY, testY) = train_test_split(new_fe,labels,
	test_size=0.25 , random_state=42)
print(trainY.shape)

(8311, 100)


In [5]:
trainX = tf.keras.utils.normalize(trainX, axis=1)  # scales data between 0 and 1
testX = tf.keras.utils.normalize(testX, axis=1)

In [12]:
model1 = keras.Sequential()
model1.add(keras.Input(shape=(7,72)))
model1.add(LSTM(32))
model1.add(Dense(8, activation="relu"))
model1.add(Dense(4, activation="relu"))
model1.add(Dense(1))

In [13]:

model1.compile(loss='mean_squared_error', optimizer='adam' ,metrics=["accuracy"])



history = model1.fit(trainX, trainY, epochs=10, validation_split=0.2)

test_scores = model1.evaluate(testX, testY, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/10
208/208 [==============================] - 4s 7ms/step - loss: 0.0117 - accuracy: 0.9900 - val_loss: 0.0099 - val_accuracy: 0.9900
Epoch 2/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0099 - accuracy: 0.9900 - val_loss: 0.0099 - val_accuracy: 0.9900
Epoch 3/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0099 - accuracy: 0.9900 - val_loss: 0.0099 - val_accuracy: 0.9900
Epoch 4/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0099 - accuracy: 0.9900 - val_loss: 0.0099 - val_accuracy: 0.9900
Epoch 5/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0099 - accuracy: 0.9900 - val_loss: 0.0099 - val_accuracy: 0.9900
Epoch 6/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0099 - accuracy: 0.9900 - val_loss: 0.0099 - val_accuracy: 0.9900
Epoch 7/10
208/208 [==============================] - 1s 4ms/step - loss: 0.0099 - accuracy: 0.9900 - val_loss: 0.0099 - val_accuracy: 0.9900
Epoch 

In [11]:
myModel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 7, 72)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                13440     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 13,473
Trainable params: 13,473
Non-trainable params: 0
_________________________________________________________________


In [12]:

myModel.save('F:/Lstm_model/')

INFO:tensorflow:Assets written to: F:/Lstm_model/assets


INFO:tensorflow:Assets written to: F:/Lstm_model/assets


In [13]:
new_model = tf.keras.models.load_model('F:/Lstm_model/')

# Check its architecture
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 7, 72)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                13440     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 13,473
Trainable params: 13,473
Non-trainable params: 0
_________________________________________________________________
